# Use Stripe with agents.json
This notebook demonstrates how to use agents.json to interact with the Stripe API. This simple example shows loading the agents.json file and using it to execute Stripe tasks.


In [1]:
%pip install agentsjson
%pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Setup API keys and references to `agents.json` files

In [2]:
THEODDS_API_KEY="<your-theodds-api-key>"
OPENAI_API_KEY="<your-openai-api-key>" 
agents_json_url = "https://gist.githubusercontent.com/yagnyaPatel/1df27ef8d5a853f7bf7372e4fb761a75/raw/1d9ca63406113aa1e8ac2d800e14a71c895b54e5/theodds.agents.json"

### Load the agents.json file

In [3]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
import agentsjson.core as core

# load the agents.json file
data: Bundle = core.load_agents_json(agents_json_url)
flows = data.agentsJson.flows

### Define the Agent

In [4]:
query = "What the score for the last Lakers game"

In [5]:
from agentsjson.core import ToolFormat

# Format the flows data for the prompt
flows_context = core.flows_prompt(flows)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Stripe API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [6]:
from agentsjson.core.models.auth import AuthType, ApiKeyAuthConfig
auth = ApiKeyAuthConfig(type=AuthType.API_KEY, key_value=THEODDS_API_KEY)

In [7]:
core.flows_tools(flows, format=ToolFormat.OPENAI)

[{'type': 'function',
  'function': {'name': 'theodds_get_v4_sports_flow',
   'description': "Retrieve a list of available sports and tournaments from The Odds API. Include your API key and optionally specify the 'all' flag to include all sports.",
   'parameters': {'type': 'object',
    'properties': {'parameters': {'type': 'object',
      'properties': {'apiKey': {'type': 'string',
        'description': 'Your API key for The Odds API.'},
       'all': {'type': 'string',
        'description': 'Optional flag to include all sports even those not currently in-season.'}},
      'required': ['apiKey']}},
    'required': ['parameters']},
   'additionalProperties': False}},
 {'type': 'function',
  'function': {'name': 'theodds_get_v4_sports_odds_flow',
   'description': 'Retrieve sportsbook odds for a specified sport. Provide the sport key, API key, regions, and optional parameters for markets and formatting.',
   'parameters': {'type': 'object',
    'properties': {'parameters': {'type': '

### Run the agent

In [8]:
from openai import OpenAI
from agentsjson.core.executor import execute_flows
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=core.flows_tools(flows, format=ToolFormat.OPENAI),
    temperature=0
)

response = execute_flows(response, format=core.ToolFormat.OPENAI, bundle=data, flows=flows, auth=auth)

response

{'theodds_get_v4_sports_flow': {'success': [{'key': 'americanfootball_ncaaf', 'group': 'American Football', 'title': 'NCAAF', 'description': 'US College Football', 'active': True, 'has_outrights': False}, {'key': 'aussierules_afl', 'group': 'Aussie Rules', 'title': 'AFL', 'description': 'Aussie Football', 'active': True, 'has_outrights': False}, {'key': 'baseball_mlb_world_series_winner', 'group': 'Baseball', 'title': 'MLB World Series Winner', 'description': 'World Series Winner 2025', 'active': True, 'has_outrights': True}, {'key': 'baseball_ncaa', 'group': 'Baseball', 'title': 'NCAA Baseball', 'description': 'US College Baseball', 'active': True, 'has_outrights': False}, {'key': 'basketball_euroleague', 'group': 'Basketball', 'title': 'Basketball Euroleague', 'description': 'Basketball Euroleague', 'active': True, 'has_outrights': False}, {'key': 'basketball_nba', 'group': 'Basketball', 'title': 'NBA', 'description': 'US Basketball', 'active': True, 'has_outrights': False}, {'key': 